In [1]:
from api_key import client_secret
from api_key import client_id
from api_key import api_key
from suburb_list import suburb_list_melbourne
from suburb_list import values
import requests
import json
import gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
import csv

ModuleNotFoundError: No module named 'api_key'

## Get the postcode of each suburb 

In [10]:
scopes = ['api_addresslocators_read']
auth_url = 'https://auth.domain.com.au/v1/connect/token'
url_endpoint = 'https://api.domain.com.au/sandbox/v1/addressLocators/'
suburb_id = []
postcode_list = []
response = requests.post(auth_url, data = {'client_id':client_id,
                                               'client_secret':client_secret,
                                               'grant_type':'client_credentials',
                                               'scope':scopes,
                                               'Content-Type':'text/json'
                                              })
json_res = response.json()
access_token=json_res['access_token']
for suburb in suburb_list_melbourne:
    params = {"searchLevel" : "Suburb",
             "suburb" : suburb,
             "state" : "vic"}
    #print(access_token)
    auth = {'Authorization':'Bearer ' + access_token}
    res1 = requests.get(url_endpoint, headers=auth , params = params)    
    try:
        r = res1.json()
        print(f"there is correct suburb id in {suburb}")
        suburb_id.append(r[0]["ids"][0]['id'])
        postcode_list.append(r[0]["addressComponents"][1]['shortName'])
    except:
        print(f"there is no suburb id in {suburb}")
        suburb_id.append("N/A")
        postcode_list.append("N/A")


there is correct suburb id in Abbotsford
there is correct suburb id in Aberfeldie
there is correct suburb id in Airport West
there is correct suburb id in Albanvale
there is correct suburb id in Albert Park
there is correct suburb id in Albion
there is correct suburb id in Alphington
there is correct suburb id in Altona Meadows
there is correct suburb id in Altona North
there is correct suburb id in Altona
there is correct suburb id in Ardeer
there is correct suburb id in Armadale
there is correct suburb id in Arthurs Seat
there is correct suburb id in Ascot Vale
there is correct suburb id in Ashburton
there is correct suburb id in Ashwood
there is correct suburb id in Aspendale
there is correct suburb id in Aspendale Gardens
there is correct suburb id in Attwood
there is correct suburb id in Auburn
there is no suburb id in Auburn
there is correct suburb id in Aurora
there is no suburb id in Aurora
there is correct suburb id in Avondale Heights
there is correct suburb id in Avonsleigh


In [12]:
city_id = pd.DataFrame({"Suburb" : suburb_list_melbourne,
                        "ID_Domain" : suburb_id,
                       "Postcode" : postcode_list})
city_id.to_csv("Resources/city_id_postcode.csv")

In [34]:
city_id_clean = pd.read_csv("Resources/city_id_postcode.csv").dropna()
city_id_clean = city_id_clean.iloc[: , 1:]
city_id_clean['ID_Domain'] = city_id_clean['ID_Domain'].astype(int)
city_id_clean['Postcode'] = city_id_clean['Postcode'].astype(int)
city_id_clean['Postcode'] = city_id_clean['Postcode'].astype(str)
suburb_dataset = city_id_clean.set_index('Postcode')
suburb_dataset.head()

Suburb  ID_Domain
Postcode                         
3067        Abbotsford        627
3040        Aberfeldie        647
3042      Airport West        757
3021         Albanvale        787
3206       Albert Park        797

In [55]:
yearly_data_2019 = suburb_dataset.copy()
yearly_data_2020 = suburb_dataset.copy()
yearly_data_2021 = suburb_dataset.copy()
yearly_data_2021.head()

Suburb  ID_Domain
Postcode                         
3067        Abbotsford        627
3040        Aberfeldie        647
3042      Airport West        757
3021         Albanvale        787
3206       Albert Park        797

In [38]:
postcode_list_clean = city_id_clean["Postcode"].tolist()
postcode_list_clean

['3067',
 '3040',
 '3042',
 '3021',
 '3206',
 '3020',
 '3078',
 '3028',
 '3025',
 '3018',
 '3022',
 '3143',
 '3936',
 '3032',
 '3147',
 '3147',
 '3195',
 '3195',
 '3049',
 '3034',
 '3782',
 '3183',
 '3103',
 '3104',
 '3175',
 '3911',
 '3153',
 '3153',
 '3807',
 '3193',
 '3160',
 '3160',
 '3160',
 '3081',
 '3204',
 '3165',
 '3806',
 '3918',
 '3193',
 '3130',
 '3130',
 '3130',
 '3942',
 '3196',
 '3155',
 '3128',
 '3129',
 '3128',
 '3195',
 '3019',
 '3088',
 '3186',
 '3187',
 '3047',
 '3338',
 '3012',
 '3056',
 '3057',
 '3055',
 '3428',
 '3105',
 '3083',
 '3121',
 '3023',
 '3023',
 '3125',
 '3151',
 '3023',
 '3037',
 '3124',
 '3061',
 '3126',
 '3054',
 '3053',
 '3163',
 '3023',
 '3197',
 '3201',
 '3162',
 '3145',
 '3161',
 '3162',
 '3148',
 '3196',
 '3196',
 '3192',
 '3116',
 '3169',
 '3168',
 '3169',
 '3782',
 '3068',
 '3058',
 '3058',
 '3030',
 '3770',
 '3066',
 '3048',
 '3064',
 '3977',
 '3977',
 '3977',
 '3977',
 '3977',
 '3121',
 '3919',
 '3136',
 '3136',
 '3136',
 '3136',
 '3047',
 

## Get the yearly data of property

In [56]:
#create a list to restore the number
scopes = ['api_suburbperformance_read']
auth_url = 'https://auth.domain.com.au/v1/connect/token'
response = requests.post(auth_url, data = {
                            'client_id':client_id,
                            'client_secret':client_secret,
                            'grant_type':'client_credentials',
                            'scope':scopes,
                            'Content-Type':'text/json'
                            })
json_res = response.json()
access_token=json_res['access_token']
auth = {'Authorization':'Bearer ' + access_token}
i = 0
for suburb in suburb_dataset['Suburb']:
    postcode_number = postcode_list_clean[i]
    print(postcode_number)
    url_sale = f'https://api.domain.com.au/sandbox/v2/suburbPerformanceStatistics/VIC/{suburb}/{postcode_number}?'
    params = {"propertyCategory" : "House",
             "periodSize" : "years",
             "startingPeriodRelativeToCurrent":1,
             "totalPeriods":3}
    res = requests.get(url_sale, headers=auth, params = params)
    try:
        s = res.json()
    #s["series"]["seriesInfo"][0]["values"] represent 2019
    #s["series"]["seriesInfo"][1]["values"] represent 2020
    #s["series"]["seriesInfo"][2]["values"] represent 2021
        for value in values:
            try:
                yearly_data_2019.loc[postcode_number,value] = s["series"]["seriesInfo"][0]["values"][value]
                yearly_data_2020.loc[postcode_number,value] = s["series"]["seriesInfo"][1]["values"][value]
                yearly_data_2021.loc[postcode_number,value] = s["series"]["seriesInfo"][2]["values"][value]
                print(f"there is data with in {suburb} of {value}")
            except JSONDecodeError:
                print(f"there is no value in {suburb} of {value}")
                yearly_data_2019.loc[postcode_number,value] = "N/A"
                yearly_data_2020.loc[postcode_number,value] = "N/A"
                yearly_data_2021.loc[postcode_number,value] = "N/A"
    except:
        print(f"there is problem with {suburb} in url")
    i = i + 1

3067
there is data with in Abbotsford of medianSoldPrice
there is data with in Abbotsford of numberSold
there is data with in Abbotsford of highestSoldPrice
there is data with in Abbotsford of lowestSoldPrice
there is data with in Abbotsford of 5thPercentileSoldPrice
there is data with in Abbotsford of 25thPercentileSoldPrice
there is data with in Abbotsford of 75thPercentileSoldPrice
there is data with in Abbotsford of 95thPercentileSoldPrice
there is data with in Abbotsford of medianSaleListingPrice
there is data with in Abbotsford of numberSaleListing
there is data with in Abbotsford of highestSaleListingPrice
there is data with in Abbotsford of lowestSaleListingPrice
there is data with in Abbotsford of auctionNumberAuctioned
there is data with in Abbotsford of auctionNumberSold
there is data with in Abbotsford of medianRentListingPrice
there is data with in Abbotsford of numberRentListing
there is data with in Abbotsford of highestRentListingPrice
there is data with in Abbotsford o

In [60]:
#raw dataset
yearly_data_2019.to_csv("Resources/yearly_data_2019.csv")
yearly_data_2020.to_csv("Resources/yearly_data_2020.csv")
yearly_data_2021.to_csv("Resources/yearly_data_2021.csv")

In [59]:
#clean dataset and save to csv file
yearly_data_2019_clean = yearly_data_2019.dropna()
yearly_data_2020_clean = yearly_data_2020.dropna()
yearly_data_2021_clean = yearly_data_2021.dropna()
yearly_data_2019_clean.to_csv("Resources/yearly_data_clean_2019.csv")
yearly_data_2020_clean.to_csv("Resources/yearly_data_clean_2020.csv")
yearly_data_2021_clean.to_csv("Resources/yearly_data_clean_2021.csv")

## Get the quartly data of property

In [62]:
quarter_data_2019_1st = suburb_dataset.copy()
quarter_data_2019_2nd = suburb_dataset.copy()
quarter_data_2019_3rd = suburb_dataset.copy()
quarter_data_2019_4th = suburb_dataset.copy()
quarter_data_2020_1st = suburb_dataset.copy()
quarter_data_2020_2nd = suburb_dataset.copy()
quarter_data_2020_3rd = suburb_dataset.copy()
quarter_data_2020_4th = suburb_dataset.copy()
quarter_data_2021_1st = suburb_dataset.copy()
quarter_data_2021_2nd = suburb_dataset.copy()
quarter_data_2019_1st.head()

Suburb  ID_Domain
Postcode                         
3067        Abbotsford        627
3040        Aberfeldie        647
3042      Airport West        757
3021         Albanvale        787
3206       Albert Park        797

In [63]:
#create a list to restore the number
scopes = ['api_suburbperformance_read']
auth_url = 'https://auth.domain.com.au/v1/connect/token'
response = requests.post(auth_url, data = {
                            'client_id':client_id,
                            'client_secret':client_secret,
                            'grant_type':'client_credentials',
                            'scope':scopes,
                            'Content-Type':'text/json'
                            })
json_res = response.json()
access_token=json_res['access_token']
auth = {'Authorization':'Bearer ' + access_token}
i = 0
for suburb in suburb_dataset['Suburb']:
    postcode_number = postcode_list_clean[i]
    print(postcode_number)
    url_sale = f'https://api.domain.com.au/sandbox/v2/suburbPerformanceStatistics/VIC/{suburb}/{postcode_number}?'
    params = {"propertyCategory" : "House",
             "periodSize" : "quarters",
             "startingPeriodRelativeToCurrent":1,
             "totalPeriods":10}
    res = requests.get(url_sale, headers=auth, params = params)
    try:
        s = res.json()
    #s["series"]["seriesInfo"][0]["values"] represent 2019 1st
    #s["series"]["seriesInfo"][1]["values"] represent 2019 2nd
    #s["series"]["seriesInfo"][3]["values"] represent 2019 3rd
    #s["series"]["seriesInfo"][4]["values"] represent 2019 4th
        for value in values:
            try:
                quarter_data_2019_1st.loc[postcode_number,value] = s["series"]["seriesInfo"][0]["values"][value]
                quarter_data_2019_2nd.loc[postcode_number,value] = s["series"]["seriesInfo"][1]["values"][value]
                quarter_data_2019_3rd.loc[postcode_number,value] = s["series"]["seriesInfo"][2]["values"][value]
                quarter_data_2019_4th.loc[postcode_number,value] = s["series"]["seriesInfo"][3]["values"][value]
                quarter_data_2020_1st.loc[postcode_number,value] = s["series"]["seriesInfo"][4]["values"][value]
                quarter_data_2020_2nd.loc[postcode_number,value] = s["series"]["seriesInfo"][5]["values"][value]
                quarter_data_2020_3rd.loc[postcode_number,value] = s["series"]["seriesInfo"][6]["values"][value]
                quarter_data_2020_4th.loc[postcode_number,value] = s["series"]["seriesInfo"][7]["values"][value]
                quarter_data_2021_1st.loc[postcode_number,value] = s["series"]["seriesInfo"][8]["values"][value]
                quarter_data_2021_2nd.loc[postcode_number,value] = s["series"]["seriesInfo"][9]["values"][value]
                print(f"there is data with in {suburb} of {value}")
            except JSONDecodeError:
                print(f"there is no value in {suburb} of {value}")
    except:
        print(f"there is problem with {suburb} in url")
    i = i + 1

3067
there is data with in Abbotsford of medianSoldPrice
there is data with in Abbotsford of numberSold
there is data with in Abbotsford of highestSoldPrice
there is data with in Abbotsford of lowestSoldPrice
there is data with in Abbotsford of 5thPercentileSoldPrice
there is data with in Abbotsford of 25thPercentileSoldPrice
there is data with in Abbotsford of 75thPercentileSoldPrice
there is data with in Abbotsford of 95thPercentileSoldPrice
there is data with in Abbotsford of medianSaleListingPrice
there is data with in Abbotsford of numberSaleListing
there is data with in Abbotsford of highestSaleListingPrice
there is data with in Abbotsford of lowestSaleListingPrice
there is data with in Abbotsford of auctionNumberAuctioned
there is data with in Abbotsford of auctionNumberSold
there is data with in Abbotsford of medianRentListingPrice
there is data with in Abbotsford of numberRentListing
there is data with in Abbotsford of highestRentListingPrice
there is data with in Abbotsford o

In [65]:
quarter_data_2019_1st.head()

Suburb  ID_Domain  medianSoldPrice  numberSold  \
Postcode                                                         
3067        Abbotsford        627         990000.0        11.0   
3040        Aberfeldie        647        1300000.0        53.0   
3042      Airport West        757         950000.0        15.0   
3021         Albanvale        787         600000.0        82.0   
3206       Albert Park        797        2475000.0        10.0   

          highestSoldPrice  lowestSoldPrice  5thPercentileSoldPrice  \
Postcode                                                              
3067             1650000.0         790000.0                790000.0   
3040             2980000.0         297000.0                447000.0   
3042             1500000.0         592000.0                592000.0   
3021             1281000.0         345000.0                386000.0   
3206             5500000.0        1353000.0               1353000.0   

          25thPercentileSoldPrice  75thPercentileSoldPrice  \
Postcode                                                     
3067                     860000.0                1450000.0   
3040                     658000.0                1740000.0   
3042                     800000.0                1135000.0   
3021                     520000.0                 656000.0   
3206                    1865000.0                2800000.0   

          95thPercentileSoldPrice  medianSaleListingPrice  numberSaleListing  \
Postcode                                                                       
3067                    1650000.0               1190000.0               27.0   
3040                    2559000.0               1339000.0              109.0   
3042                    1500000.0                950000.0               36.0   
3021                     820000.0                600000.0              268.0   
3206                    5500000.0               2787000.0               15.0   

          highestSaleListingPrice  lowestSaleListingPrice  \
Postcode                                                    
3067                    1750000.0                115000.0   
3040                    5000000.0                280000.0   
3042                    1400000.0                650000.0   
3021                    2500000.0                 80000.0   
3206                   13000000.0               1890000.0   

          auctionNumberAuctioned  auctionNumberSold  medianRentListingPrice  \
Postcode                                                                      
3067                        13.0                4.0                   650.0   
3040                        36.0               12.0                   540.0   
3042                        14.0                4.0                   545.0   
3021                        34.0                9.0                   360.0   
3206                         8.0                3.0                   850.0   

          numberRentListing  highestRentListingPrice  lowestRentListingPrice  
Postcode                                                                      
3067                   49.0                   1250.0                   400.0  
3040                  140.0                   1400.0                   290.0  
3042                   45.0                   1350.0                   270.0  
3021                  169.0                   1200.0                   190.0  
3206                   31.0                   1700.0                   475.0

In [67]:
quarter_data_2019_3rd.head()

Suburb  ID_Domain  medianSoldPrice  numberSold  \
Postcode                                                         
3067        Abbotsford        627        1030000.0        13.0   
3040        Aberfeldie        647        1303000.0        46.0   
3042      Airport West        757        1087000.0        27.0   
3021         Albanvale        787         573000.0        92.0   
3206       Albert Park        797              NaN         5.0   

          highestSoldPrice  lowestSoldPrice  5thPercentileSoldPrice  \
Postcode                                                              
3067             1570000.0         845000.0                845000.0   
3040             6500000.0         480000.0                615000.0   
3042             1744000.0         615000.0                816000.0   
3021              985000.0         407000.0                430000.0   
3206            12300000.0        1950000.0                     NaN   

          25thPercentileSoldPrice  75thPercentileSoldPrice  \
Postcode                                                     
3067                     930000.0                1165000.0   
3040                     850000.0                1681000.0   
3042                     965000.0                1200000.0   
3021                     506000.0                 600000.0   
3206                          NaN                      NaN   

          95thPercentileSoldPrice  medianSaleListingPrice  numberSaleListing  \
Postcode                                                                       
3067                    1570000.0               1100000.0               23.0   
3040                    2370000.0               1450000.0               81.0   
3042                    1560000.0               1013000.0               46.0   
3021                     740000.0                580000.0              271.0   
3206                          NaN               2920000.0               11.0   

          highestSaleListingPrice  lowestSaleListingPrice  \
Postcode                                                    
3067                    2468000.0                115000.0   
3040                    3450000.0                400000.0   
3042                    1470000.0                580000.0   
3021                    1418000.0                 62000.0   
3206                   13000000.0               1990000.0   

          auctionNumberAuctioned  auctionNumberSold  medianRentListingPrice  \
Postcode                                                                      
3067                         8.0                6.0                   650.0   
3040                        24.0               15.0                   500.0   
3042                        13.0                9.0                   550.0   
3021                        24.0                7.0                   360.0   
3206                         3.0                1.0                   820.0   

          numberRentListing  highestRentListingPrice  lowestRentListingPrice  
Postcode                                                                      
3067                   47.0                   1100.0                   415.0  
3040                  140.0                   1500.0                   290.0  
3042                   38.0                   1150.0                   240.0  
3021                  200.0                   1517.0                   250.0  
3206                   29.0                   2200.0                   360.0

In [70]:
#raw dataset
quarter_data_2019_1st.to_csv("Resources/quarter_data_2019_1st.csv")
quarter_data_2019_2nd.to_csv("Resources/quarter_data_2019_2nd.csv")
quarter_data_2019_3rd.to_csv("Resources/quarter_data_2019_3rd.csv")
quarter_data_2019_4th.to_csv("Resources/quarter_data_2019_4th.csv")
quarter_data_2020_1st.to_csv("Resources/quarter_data_2020_1st.csv")
quarter_data_2020_2nd.to_csv("Resources/quarter_data_2020_2nd.csv")
quarter_data_2020_3rd.to_csv("Resources/quarter_data_2020_3rd.csv")
quarter_data_2020_4th.to_csv("Resources/quarter_data_2020_4th.csv")
quarter_data_2021_1st.to_csv("Resources/quarter_data_2021_1st.csv")
quarter_data_2021_2nd.to_csv("Resources/quarter_data_2021_2nd.csv")

In [69]:
quarter_data_2019_1st_clean = quarter_data_2019_1st.dropna()
quarter_data_2019_2nd_clean = quarter_data_2019_2nd.dropna()
quarter_data_2019_3rd_clean = quarter_data_2019_3rd.dropna()
quarter_data_2019_4th_clean = quarter_data_2019_4th.dropna()
quarter_data_2020_1st_clean = quarter_data_2020_1st.dropna()
quarter_data_2020_2nd_clean = quarter_data_2020_2nd.dropna()
quarter_data_2020_3rd_clean = quarter_data_2020_3rd.dropna()
quarter_data_2020_4th_clean = quarter_data_2020_4th.dropna()
quarter_data_2021_1st_clean = quarter_data_2021_1st.dropna()
quarter_data_2021_2nd_clean = quarter_data_2021_2nd.dropna()
#to csv
quarter_data_2019_1st_clean.to_csv("Resources/quarter_data_2019_1st_clean.csv")
quarter_data_2019_2nd_clean.to_csv("Resources/quarter_data_2019_2nd_clean.csv")
quarter_data_2019_3rd_clean.to_csv("Resources/quarter_data_2019_3rd_clean.csv")
quarter_data_2019_4th_clean.to_csv("Resources/quarter_data_2019_4th_clean.csv")
quarter_data_2020_1st_clean.to_csv("Resources/quarter_data_2020_1st_clean.csv")
quarter_data_2020_2nd_clean.to_csv("Resources/quarter_data_2020_2nd_clean.csv")
quarter_data_2020_3rd_clean.to_csv("Resources/quarter_data_2020_3rd_clean.csv")
quarter_data_2020_4th_clean.to_csv("Resources/quarter_data_2020_4th_clean.csv")
quarter_data_2021_1st_clean.to_csv("Resources/quarter_data_2021_1st_clean.csv")
quarter_data_2021_2nd_clean.to_csv("Resources/quarter_data_2021_2nd_clean.csv")